# Grafo de agentes

In [ ]:
%%capture
!pip install pyvis
!pip install igraph
!apt install libgraphviz-dev
!pip install pygraphviz
!pip install chart_studio

In [ ]:
import os
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from pyvis import network as net
import igraph as ig
import chart_studio.plotly as py
import plotly.graph_objects as go
from plotly.offline import iplot
from matplotlib import font_manager as fm

In [ ]:
# Definições gerais
outdir = './grafos_anon_png_mar24'

In [ ]:
%%capture
!wget https://github.com/google/fonts/raw/main/ofl/firasans/FiraSans-Regular.ttf
!wget https://github.com/google/fonts/raw/main/ofl/firasans/FiraSans-Bold.ttf
!wget https://github.com/google/fonts/raw/main/ofl/firasans/FiraSans-Medium.ttf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

if not os.getcwd().endswith('{Nome da pasta no drive}'):
    os.chdir('/content/drive/{Nome da pasta no drive}')

# Import fonts
font_files = fm.findSystemFonts('.')

for font_file in font_files:
    fm.fontManager.addfont(font_file)

Mounted at /content/drive


In [ ]:
# Planilha do google sheets
sheet_id = '' # Inserir id do link da planilha-modelo preenchida

In [ ]:
# Leitura dos estilos
sheet_name = 'estilos'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_estilo = pd.read_csv(url)
df_estilo = df_estilo[[i for i in df_estilo.columns if not i.startswith('Unnamed:')]]
df_estilo = df_estilo.rename(columns={'cor_preenchimento':'face', 'cor_borda':'border', 'espessura_borda':'edgewidth', 'cor_fonte':'label'}).fillna('').set_index('tag')
color_by_tags = df_estilo.to_dict(orient='index')

In [ ]:
# Função para formatar um texto com quebras de linha conforme um tamanho limite
def break_text(texto, limite=12, breaker='\n'):
    if not isinstance(texto, str):
        texto = str(texto)
    line_len = 0
    texto_out = ''
    for i in texto.split(' '):
        if texto_out == '':
            texto_out += i
            continue

        len_last_line = len(texto_out.split(breaker)[-1])
        len_add = len(i)

        if len_last_line+len_add < limite:
            texto_out += (' '+i)
        else:
            texto_out += (breaker+i)

    return texto_out.strip().replace(f' {breaker}',breaker)

In [ ]:
class grafo_atores:
    def __init__(self, vinculos, agentes, lugares, foco=[]):
        self.grafo = nx.Graph()
        self.foco = foco
        self.vinculos = vinculos
        self.agentes = agentes
        self.lugares = lugares

        # Adiciona as arestas
        for v in vinculos.to_dict(orient='records'):
            self.grafo.add_edge(v['card_1'], v['card_2'], data={'anotacao':v['anotacao'], 'territorios':v['territorio']})

        # Preenche dados para agentes
        node_data = agentes.set_index('card').to_dict(orient='index')
        for k,v in node_data.items():
            v['tipo']='agente'

        nx.set_node_attributes(self.grafo, node_data)

        # Preenche dados para processos
        proc_dic = {'tipo':'processo', 'escala':2, 'escala':2, 'tags':'proc'}
        lugar_dic = {'tipo':'lugar', 'escala':2, 'escala':2, 'tags':'lugar'}
        proc_data = {i:proc_dic for i in self.grafo.nodes if i not in node_data}
        lugar_data = {i:lugar_dic for i in self.grafo.nodes if i in proc_data and i in self.lugares['lugar'].unique()}
        proc_data.update(lugar_data)
        nx.set_node_attributes(self.grafo, proc_data)

    def selecionar_nos(self, origin, search_dist=1, how='dict'):
        if isinstance(origin, str):
            origin = [origin]
        nodes = {0:origin}
        level = 0

        while level < search_dist:
            nodes[level+1] = []
            past_nodes = [i for j in nodes.values() for i in j]
            for n in nodes[level]:
                nodes_to_add = [i for i in dict(self.grafo[n]).keys() if i not in past_nodes]
                nodes[level+1] += nodes_to_add
            level += 1

        if how=='dict':
            return nodes
        elif how=='list':
            return [i for j in nodes.values() for i in j]
        else:
            raise Exception

    def subgrafo_do_no(self, ref_nodes=[], search_dist=1):
        if isinstance(ref_nodes, str):
            ref_nodes = [ref_nodes]
        if not ref_nodes: ref_nodes = self.foco
        list_of_nodes = self.selecionar_nos(ref_nodes, search_dist=search_dist, how='list')
        subgraph = self.grafo.subgraph(list_of_nodes)

        sub_agentes = self.agentes[self.agentes['card'].apply(lambda x: x in list_of_nodes)]
        sub_vinculos = self.vinculos[self.vinculos.apply(lambda x: x['card_1'] in list_of_nodes and x['card_2'] in list_of_nodes, axis=1)]
        sub_lugares = self.lugares[self.lugares['lugar'].apply(lambda x: x in list_of_nodes)]

        return grafo_atores(sub_vinculos, sub_agentes, sub_lugares, foco=ref_nodes)


    def plot(self, title='', figsize=(25,18), savename='', savedir=''):
        pos = nx.drawing.nx_agraph.graphviz_layout(self.grafo, prog='neato')
        #pos = nx.kamada_kawai_layout(self.grafo)
        fig = plt.figure(1, figsize=figsize, dpi=100, facecolor='white')
        fig.tight_layout()

        # Desenha as arestas
        for edge in self.grafo.edges():
            source, target = edge
            rad = 0.2
            con_style = f"arc3,rad={rad}"

            t1 = self.grafo.nodes[target]['tipo']
            t2 = self.grafo.nodes[source]['tipo']
            p1 = '<' if t1 in ['processo', 'lugar'] else ''
            p2 = '>' if t2 in ['processo', 'lugar'] else ''
            tag1 = self.grafo.nodes[target]['tags']
            tag2 = self.grafo.nodes[source]['tags']

            if tag1 == tag2:
                linecolor = color_by_tags[tag1]['face']
            elif t1 =='processo' and t2 !='processo':
                linecolor = color_by_tags[tag2]['face']
            elif t1 !='processo' and t2 =='processo':
                linecolor = color_by_tags[tag1]['face']
            else:
                linecolor = 'darkgray'


            arrowstyle = f'{p1}-{p2}'
            arrowprops=dict(lw=1.5,
                            arrowstyle=arrowstyle,
                            mutation_scale=10,
                            color=linecolor,
                            alpha=0.7,
                            connectionstyle=con_style,
                            linestyle= '-',
                            shrinkA=18,
                            shrinkB=18)
            plt.annotate('',
                        xy=pos[source],
                        xytext=pos[target],
                        arrowprops=arrowprops,
                        zorder=2
                       )

        # Formato dos nós
        shape_by_tipo={'agente':'o', 'processo':'s', 'lugar':'v'}

        # Estiliza os nós conforme o tipo (agente, processo, lugar)
        for tipo, shape in shape_by_tipo.items():
            filtered_nodes = [i for i in filter(lambda x: x[1]['tipo']==tipo, self.grafo.nodes(data=True))]

            # Escala dos nós
            scale_nodes = [i[1]['escala']*300 for i in filtered_nodes]
            scale_edges = [i[1]['escala']*3 for i in filtered_nodes]

            taglist = {i[0]:i[1]['tags'] for i in filtered_nodes}

            # Estilos definidos pelo nível
            color_nodes = [color_by_tags[v]['face'] for k,v in taglist.items()]
            #color_border = [color_by_tags[v]['face'] for  k,v in taglist.items()]
            color_border = ['white' for  k,v in taglist.items()]
            edge_width = [2 for k,v in taglist.items()]

            drawn_nodes = nx.draw_networkx_nodes(self.grafo, pos, node_shape=shape, nodelist=[i[0] for i in filtered_nodes], alpha=1, node_size=scale_nodes, node_color=color_nodes)
            drawn_nodes.set_edgecolor(color_border)
            drawn_nodes.set_linewidth(edge_width)
            drawn_nodes.set_zorder(1)


        # Estiliza os textos conforme o grupo
        for group in color_by_tags:
            filtered_labels = [i for i in filter(lambda x: x[1]['tags']==group, self.grafo.nodes(data=True))]
            color_label = color_by_tags[group]['label']

            formated_names = {i[0]:break_text(i[0], 18) for i in filtered_labels}
            drawn_labels = nx.draw_networkx_labels(self.grafo, pos, formated_names, font_size=10, font_family='Fira Sans', font_weight='regular', font_color=color_label)
            #drawn_labels.set_zorder(15)

        #nx.draw_networkx_edge_labels(self.G, pos, {n: n for n in list(self.G.edges)}, font_size=6)
        plt.box(False)
        if title:
            plt.title('  '+title, color='black', fontfamily='Fira Sans', fontweight='medium', fontsize=25, loc='left', pad='-30')

        if savename:
            plt.savefig(savedir+'/'+savename+'.png', transparent=True, bbox_inches='tight')
        plt.show()


    def plot_subgrafo(self, ref_nodes, search_dist=[1], title='', figsize=(25,18), savename='', savedir=''):
        pos = nx.drawing.nx_agraph.graphviz_layout(self.grafo, prog='neato')
        fig = plt.figure(1, figsize=figsize, dpi=100, facecolor='white')
        fig.tight_layout()

        # Recorta o subgrafo para plotagem
        if isinstance(search_dist, list):
            max_search_dist = max(search_dist)
        else:
            search_dist = [search_dist]
            max_search_dist = max(search_dist)
        level_dict = self.selecionar_nos(ref_nodes, search_dist=max_search_dist, how='dict')
        subgrafo_atores = self.subgrafo_do_no(ref_nodes, max_search_dist)
        subgrafo = subgrafo_atores.grafo
        foco = subgrafo_atores.foco

        # Cria categorias de nós
        curr_level = [i for j in search_dist for i in level_dict[j]]
        prev_levels = [i for j in level_dict.values() for i in j if i not in curr_level]
        post_levels = [i for i in list(self.grafo.nodes) if i not in list(subgrafo.nodes)]

        ### Plotagem
        # Desenha as arestas do subgrafo
        for edge in self.grafo.edges():
            source, target = edge
            rad = 0.2
            con_style = f"arc3,rad={rad}"

            # Define variáveis para estilização
            t1 = self.grafo.nodes[target]['tipo']
            t2 = self.grafo.nodes[source]['tipo']
            p1 = '<' if t1 in ['processo', 'lugar'] else ''
            p2 = '>' if t2 in ['processo', 'lugar'] else ''
            tag1 = self.grafo.nodes[target]['tags']
            tag2 = self.grafo.nodes[source]['tags']

            # Define cor da aresta
            if tag1 == tag2:
                linecolor = color_by_tags[tag1]['face']
            elif t1 =='processo' and t2 !='processo':
                linecolor = color_by_tags[tag2]['face']
            elif t1 !='processo' and t2 =='processo':
                linecolor = color_by_tags[tag1]['face']
            else:
                linecolor = 'darkgray'

            # Define visibilidade
            if edge in subgrafo.edges():
                alpha = 0.7
            else:
                alpha = 0.15

            arrowstyle = f'{p1}-{p2}'
            arrowprops=dict(lw=1.5,
                            arrowstyle=arrowstyle,
                            mutation_scale=10,
                            color=linecolor,
                            alpha=alpha,
                            connectionstyle=con_style,
                            linestyle= '-',
                            shrinkA=18,
                            shrinkB=18)
            plt.annotate('',
                        xy=pos[source],
                        xytext=pos[target],
                        arrowprops=arrowprops,
                        zorder=3
                       )

        # Formato dos nós
        shape_by_tipo={'agente':'o', 'processo':'s', 'lugar':'v'}

        # Estiliza os nós conforme o tipo (agente, processo, lugar)
        for tipo, shape in shape_by_tipo.items():
            filtered_nodes = [i for i in filter(lambda x: x[1]['tipo']==tipo, subgrafo.nodes(data=True))]

            # Escala dos nós
            scale_nodes = [i[1]['escala']*300 for i in filtered_nodes]
            scale_edges = [i[1]['escala']*3 for i in filtered_nodes]

            taglist = {i[0]:i[1]['tags'] for i in filtered_nodes}

            # Estilos definidos pelo nível
            color_nodes = ['white' if k in curr_level else color_by_tags[v]['face'] for k,v in taglist.items()]
            color_border = [color_by_tags[v]['face'] if k in curr_level else 'white' for  k,v in taglist.items()]
            edge_width = [color_by_tags[v]['edgewidth'] if k in curr_level else 0 for  k,v in taglist.items()]

            drawn_nodes = nx.draw_networkx_nodes(subgrafo, pos, node_shape=shape, nodelist=[i[0] for i in filtered_nodes], alpha=1, node_size=scale_nodes, node_color=color_nodes)
            drawn_nodes.set_edgecolor(color_border)
            drawn_nodes.set_linewidth(edge_width)
            drawn_nodes.set_zorder(2)

            # Destaque no foco
            group_focus = [i for i in taglist if i in foco]
            if group_focus:
                color_focus_nodes = ['white' for i in group_focus]
                color_focus_border = ['black' for i in group_focus]
                focus_edge_width = [1 for i in group_focus]
                scale_focus_nodes = [i[1]['escala']*800 for i in filtered_nodes if i[0] in group_focus]
                focus_nodes = nx.draw_networkx_nodes(subgrafo, pos, node_shape=shape, nodelist=group_focus, alpha=1, node_size=scale_focus_nodes, node_color=color_focus_nodes)
                focus_nodes.set_edgecolor(color_focus_border)
                focus_nodes.set_linewidth(focus_edge_width)
                focus_nodes.set_zorder(1)


        # Desenha os nós posteriores
        for tipo, shape in shape_by_tipo.items():
            filtered_nodes = [i for i in filter(lambda x: x[1]['tipo']==tipo, self.grafo.nodes(data=True)) if i not in list(subgrafo.nodes)]

            # Escala dos nós
            scale_nodes = [i[1]['escala']*300 for i in filtered_nodes]
            scale_edges = [i[1]['escala']*3 for i in filtered_nodes]

            taglist = {i[0]:i[1]['tags'] for i in filtered_nodes}

            # Estilos definidos pelo nível
            color_nodes = ['white' if k in curr_level else color_by_tags[v]['face'] for k,v in taglist.items()]
            color_border = [color_by_tags[v]['face'] if k in curr_level else 'white' for  k,v in taglist.items()]
            edge_width = [color_by_tags[v]['edgewidth'] if k in curr_level else 0 for  k,v in taglist.items()]

            drawn_nodes = nx.draw_networkx_nodes(subgrafo, pos, node_shape=shape, nodelist=[i[0] for i in filtered_nodes], alpha=0.15, node_size=scale_nodes, node_color=color_nodes)
            drawn_nodes.set_edgecolor(color_border)
            drawn_nodes.set_linewidth(edge_width)
            drawn_nodes.set_zorder(1)

        line_size = 18
        # Estiliza os textos conforme o grupo, para nós em prev_levels
        for group in color_by_tags:
            filtered_labels = [i for i in filter(lambda x: x[1]['tags']==group, subgrafo.nodes(data=True))]
            filtered_labels = [i[0] for i in filtered_labels if i[0] in prev_levels and i[0] not in foco]
            color_label = color_by_tags[group]['label']

            formated_names = {i:break_text(i, line_size) for i in filtered_labels}
            drawn_labels = nx.draw_networkx_labels(subgrafo, pos, formated_names, font_size=10, font_family='Fira Sans', font_weight='regular', font_color=color_label)
            #drawn_labels.set_zorder(15)

        # Estiliza os textos conforme o grupo, para nós em curr_level
        for group in color_by_tags:
            filtered_labels = [i for i in filter(lambda x: x[1]['tags']==group, subgrafo.nodes(data=True)) if i not in foco]
            filtered_labels = [i[0] for i in filtered_labels if i[0] in curr_level]
            color_label = color_by_tags[group]['label']
            color_bbox = color_by_tags[group]['face']

            formated_names = {i:break_text(i, line_size) for i in filtered_labels}
            drawn_labels = nx.draw_networkx_labels(subgrafo, pos, formated_names, font_size=11, font_family='Fira Sans', font_weight='regular', font_color=color_label )#, bbox={'boxstyle':'round,pad=0.2,rounding_size=0.7', 'facecolor':color_bbox, 'edgecolor':'None', 'alpha':0.5})
            #drawn_labels.set_zorder(15)

        # Estiliza o texto dos focos
        if foco:
            formated_names = {i:break_text(i, line_size) for i in foco}
            drawn_labels = nx.draw_networkx_labels(subgrafo, pos, formated_names, font_size=12, font_family='Fira Sans', font_weight='medium', font_color='black')

        #nx.draw_networkx_edge_labels(self.G, pos, {n: n for n in list(self.G.edges)}, font_size=6)
        plt.title('  '+title, color='black', fontfamily='Fira Sans', fontweight='medium', fontsize=25, loc='left', pad='-30')
        plt.box(False)

        # Salva e plota
        if savename:
            plt.savefig(savedir+'/'+savename+'.png', transparent=True, bbox_inches='tight')
        plt.show()

## Importação

In [ ]:
# Leitura dos rótulos
sheet_name = 'rotulos'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_rotulos = pd.read_csv(url)
dic_rotulos = df_rotulos.to_dict(orient='records')
dic_rotulos = {i['card']:i['rotulo'] for i in dic_rotulos}

# Leitura dos agentes
sheet_name = 'agentes'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_agentes = pd.read_csv(url)
df_agentes = df_agentes[[i for i in df_agentes.columns if not i.startswith('Unnamed:')]]
df_agentes['tags'] = df_agentes['tags'].fillna('').apply(lambda x: list(set([i.strip() for i in x.split(';') if i.strip()])))
df_agentes['tags'] = df_agentes['tags'].apply(lambda x: x[0] if len(x)==1 else '')

# Leitura dos vínculos
sheet_name = 'vinculos'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_vinculos = pd.read_csv(url)
df_vinculos = df_vinculos[[i for i in df_vinculos.columns if not i.startswith('Unnamed:')]].dropna(subset=['card_1','card_1'])
df_vinculos['territorio'] = df_vinculos['territorio'].fillna('').apply(lambda x: list(set([i.strip() for i in x.split(';') if i.strip()])))

# Leitura dos lugares
sheet_name = 'lugares'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_lugares = pd.read_csv(url)
df_lugares = df_lugares[[i for i in df_lugares.columns if not i.startswith('Unnamed:')]]
df_lugares['tipo'] = df_agentes['tipo'].fillna('')

# Atualiza conforme rótulos
df_agentes['card'] = df_agentes['card'].apply(lambda x: dic_rotulos[x] if x in dic_rotulos else x)
df_vinculos['card_1'] = df_vinculos['card_1'].apply(lambda x: dic_rotulos[x] if x in dic_rotulos else x)
df_vinculos['card_2'] = df_vinculos['card_2'].apply(lambda x: dic_rotulos[x] if x in dic_rotulos else x)

## Plotagem 2D

In [ ]:
J = grafo_atores(df_vinculos, df_agentes, df_lugares)
J.plot(figsize=(22,22), savename='Geral', savedir=outdir)

In [ ]:
temas = {
    '':[''],
} # Inserir lista no formato 'titulo':['card', 'card', 'card']

for nome_tema, cards in temas.items():
    for level in [1,2]:
        savename = f'{nome_tema}_n{level}'
        J.plot_subgrafo(ref_nodes=cards,
                        search_dist=[level],
                        figsize=(22,22),
                        savename=savename,
                        savedir=outdir)

Output hidden; open in https://colab.research.google.com to view.

## Plotagem 3D

In [ ]:
iG = ig.Graph.TupleList(J.grafo.edges(), directed=False)

gdata = iG.get_vertex_dataframe().reset_index().to_dict(orient='records')
gdata = {i['vertex ID']:J.grafo.nodes[i['name']] for i in gdata}

labels = iG.get_vertex_dataframe()['name'].to_list()
#labels = [break_text(i, 18, '<br>') for i in labels]

#color_by_tipo={'agente':'tomato', 'processo':'slategray', 'lugar':'dodgerblue'}
shape_by_tipo={'agente':'circle', 'processo':'square', 'lugar':'diamond'}
group_color = iG.get_vertex_dataframe()['name'].apply(lambda x: color_by_tags[J.grafo.nodes[x]['tags']]['face']).to_list()
group_border = iG.get_vertex_dataframe()['name'].apply(lambda x: color_by_tags[J.grafo.nodes[x]['tags']]['border']).to_list()
group_shape = iG.get_vertex_dataframe()['name'].apply(lambda x: shape_by_tipo[J.grafo.nodes[x]['tipo']]).to_list()
size = iG.get_vertex_dataframe()['name'].apply(lambda x: 8*J.grafo.nodes[x]['escala']).fillna(0).to_list()

layt=iG.layout('kk', dim=3)
N = iG.vcount()

Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
edge_color = []

for e1, e2 in iG.get_edgelist():
    Xe+=[layt[e1][0],layt[e2][0], None]# x-coordinates of edge ends
    Ye+=[layt[e1][1],layt[e2][1], None]
    Ze+=[layt[e1][2],layt[e2][2], None]

    tag1 = gdata[e1]['tags']
    tag2 = gdata[e2]['tags']
    t1 = gdata[e1]['tipo']
    t2 = gdata[e2]['tipo']

    if tag1 == tag2:
        linecolor = color_by_tags[tag1]['face']
    elif t1 =='processo' and t2 !='processo':
        linecolor = color_by_tags[tag2]['face']
    elif t1 !='processo' and t2 =='processo':
        linecolor = color_by_tags[tag1]['face']
    else:
        linecolor = 'darkgray'

    edge_color+=[linecolor,linecolor,linecolor]

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               name='vínculos',
               line=dict(color= edge_color,
                        width=3),
               opacity=0.6,
               hoverinfo='none'
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers+text',
               name='cards',
               marker=dict(symbol=group_shape,
                             size=size,
                             color=group_color,
                             opacity=1,

                             ),
               text=labels,
               hoverinfo='text',
               textposition="middle center",
               textfont={'family':'Fira Sans Light', 'color':group_border, 'size':11},
               )

axis=dict(showbackground=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          showline=False,
          title=''
          )

layout = go.Layout(
         title="Mapa de agentes",
         width=1800,
         height=1000,
         showlegend=True,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest')

data=[trace1, trace2]
fig1=go.Figure(data=data, layout=layout)

iplot(fig1, filename='mapa-agentes-3D')
fig1.write_html(f"{outdir}/mapa-agentes-3D.html")